<a href="https://colab.research.google.com/github/jagvgithub/Algortimos_optimizacion_VIU/blob/main/Algoritmos_R3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Reto 3

## Implementar el algoritmo de búsqueda tabú para el TSP de la AG3.

Nombre: Jose Antonio Gómez Vargas

Github: [Enlace directo a Github](https://github.com/jagvgithub/Algortimos_optimizacion_VIU/blob/main/Algoritmos_R3.ipynb)

- https://github.com/jagvgithub/Algortimos_optimizacion_VIU/blob/main/Algoritmos_R3.ipynb

#Carga de librerias

In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed

---
# Carga de los datos del problema
---

In [42]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios


#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [43]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [44]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

#Funcionas basicas


In [45]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

sol_temporal = crear_solucion(Nodos)

distancia_total(sol_temporal, problem), sol_temporal

print('Creación de funciones realizado con éxito')

Creación de funciones realizado con éxito


---
# Búsqueda Aleatoria

---

In [47]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)


Mejor solución: [0, 30, 35, 20, 31, 14, 12, 13, 32, 11, 9, 24, 39, 38, 1, 10, 3, 37, 33, 34, 2, 6, 8, 18, 36, 17, 28, 40, 21, 25, 41, 23, 4, 29, 22, 19, 26, 5, 27, 16, 7, 15]
Distancia     : 3558



---
# Búsqueda Local
---

In [48]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))



Distancia Solucion Incial: 3558
Distancia Mejor Solucion Local: 3365


In [49]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

En la iteracion  47 , la mejor solución encontrada es: [0, 1, 6, 3, 27, 2, 4, 26, 5, 19, 13, 10, 8, 9, 21, 40, 24, 39, 22, 38, 30, 28, 14, 16, 15, 37, 17, 36, 35, 31, 7, 18, 12, 11, 25, 41, 23, 29, 32, 34, 33, 20]
Distancia     : 1642


---
# Recocido Simulado

---


In [50]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))

  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]


#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

In [51]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo


  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 38, 22, 41, 23, 40, 24, 21, 39, 9, 30, 33, 31, 7, 15, 16, 14, 13, 19, 5, 26, 6, 4, 12, 11, 17, 37, 36, 35, 20, 34, 32, 28, 29, 8, 10, 25, 18, 1, 3, 27, 2]
con una distancia total de 1956


---
# Implementación de búsqueda Tabú
---

In [53]:
import tsplib95
import random
import copy

# Función para generar una solución inicial aleatoria
def crear_solucion_inicial(nodos):
    solucion = nodos[:]
    random.shuffle(solucion)
    return solucion

# Función para calcular la distancia total de una solución
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += problem.get_weight(solucion[i], solucion[i+1])
    distancia_total += problem.get_weight(solucion[-1], solucion[0])  # Volver al nodo inicial
    return distancia_total

# Función para generar el vecindario (intercambiar dos nodos)
def generar_vecindario(solucion):
    vecindario = []
    for i in range(len(solucion)):
        for j in range(i + 1, len(solucion)):
            vecino = solucion[:]
            vecino[i], vecino[j] = vecino[j], vecino[i]
            vecindario.append(vecino)
    return vecindario

# Función de búsqueda tabú
def busqueda_tabu(problem, nodos, iteraciones, tamaño_tabu):
    solucion_actual = crear_solucion_inicial(nodos)
    mejor_solucion = solucion_actual[:]
    mejor_distancia = distancia_total(mejor_solucion, problem)

    lista_tabu = []

    for _ in range(iteraciones):
        vecindario = generar_vecindario(solucion_actual)
        vecino_optimo = None
        distancia_optima = float('inf')

        for vecino in vecindario:
            if vecino not in lista_tabu:
                distancia_vecino = distancia_total(vecino, problem)
                if distancia_vecino < distancia_optima:
                    vecino_optimo = vecino
                    distancia_optima = distancia_vecino

        if distancia_optima < mejor_distancia:
            mejor_solucion = vecino_optimo[:]
            mejor_distancia = distancia_optima

        solucion_actual = vecino_optimo[:]
        lista_tabu.append(solucion_actual)
        if len(lista_tabu) > tamaño_tabu:
            lista_tabu.pop(0)

    return mejor_solucion, mejor_distancia

# Cargar el problema TSP
file = "swiss42.tsp"
problem = tsplib95.load(file)

# Obtener la lista de nodos
nodos = list(problem.get_nodes())

# Realizar la búsqueda tabú
iteraciones = 1000
tamaño_tabu = 50
mejor_solucion, mejor_distancia = busqueda_tabu(problem, nodos, iteraciones, tamaño_tabu)

print("Mejor solución:", mejor_solucion)
print("Distancia de la mejor solución:", mejor_distancia)


Mejor solución: [20, 19, 13, 5, 26, 4, 6, 14, 16, 15, 37, 7, 1, 3, 0, 31, 17, 36, 35, 33, 38, 22, 39, 23, 41, 10, 18, 12, 11, 25, 40, 24, 21, 9, 8, 29, 30, 28, 2, 27, 32, 34]
Distancia de la mejor solución: 1707


---
# Entrega de resultados
---

In [55]:
import pandas as pd
from IPython.display import display

# Crear un diccionario con los datos
data = {
    'Método': ["BÚSQUEDA ALEATORIA", "BÚSQUEDA LOCAL", "RECOCIDO SIMULADO", "BUSQUEDA TABÚ"],
    'Distancia': [3558, 3365, 1956, 1707]
}

# Convertir el diccionario en un DataFrame
df = pd.DataFrame(data)

# Ordenar el DataFrame por el campo 'Distancia' de menor a mayor
df_ordenado = df.sort_values(by='Distancia')

# Definir una función para aplicar color a los valores
def color_distancia(val):
    color = 'red' if val > 3000 else 'green' if val < 2000 else 'orange'
    return f'color: {color}'

# Aplicar la función de estilo
df_styled = df_ordenado.style.applymap(color_distancia, subset=['Distancia'])

# Mostrar el DataFrame estilizado
display(df_styled)


,Método,Distancia
3,BUSQUEDA TABÚ,1707
2,RECOCIDO SIMULADO,1956
1,BÚSQUEDA LOCAL,3365
0,BÚSQUEDA ALEATORIA,3558


---
Cierre de actividad

---